In [1]:
import numpy as np
import matplotlib.pyplot as plt
import multiprocess as mp
import glob
import time
from tqdm import tqdm
import os
import sys
import pandas as pd
import dbfread 
import geopandas as gpd
import warnings
import matplotlib.pyplot as plt
import csv
import pyproj
from simpledbf import Dbf5
from datetime import datetime


In [2]:
df = pd.read_csv('/home/opengeohub/xuemeng/work_xuemeng/spatiotemporal-soc-eu/estonia/detailsed_seireandmed.csv',sep=';',low_memory=False)
column_mapping = {
    'Arvväärtus': 'Numerical value',
    'Mõõdetud arvväärtus': 'Measured numerical value',
    'Väärtuse ühik': 'Value unit',
    'Mõõdetud väärtuse ühik': 'Measured value unit',
    'Seirekoha x L-EST97': 'Monitoring location x L-EST97',
    'Seirekoha y L-EST97': 'Monitoring location y L-EST97',
    'Seireaeg': 'time',
    "Proovi sügavus (m)": "Sample depth (m)",
    "Proovivõtumeetodi proovivõtu sügavus": "Sampling method sampling depth",
    'Vastutav partner': 'Responsible partner',
    'Analüüsimeetodi  standard': 'Analysis method standard',
    'Analüüsimeetodi nimetus': 'Analysis method name',
    'Analüüsimeetodi tööjuhendi nr': 'Analysis method work instruction number',
    'Analüüsimeetodi allikas': 'Analysis method source',
    'Meetodi täpsustus': 'Method specification',
    'Näitaja nimetus': 'Indicator name'
}

df = df.rename(columns=column_mapping)

df['depth'] = np.nan
df['time'] = df['time'].str[6:10].astype(float)
df.loc[df['depth'].isna(),'depth'] = df.loc[df['depth'].isna(),'Sample depth (m)']
df.loc[df['depth'].isna(),'depth'] = df.loc[df['depth'].isna(),'Sampling method sampling depth']
df['Measured numerical value'] = df['Measured numerical value'].apply(pd.to_numeric,errors='coerce')
df['Numerical value'] = df['Numerical value'].apply(pd.to_numeric,errors='coerce')
df = df.dropna(subset = ['Measured numerical value','Numerical value'],how='all')

# aggregate every column with regard to method for later use
def concatenate_row_values(row):
    return '--'.join(row[row.notna()])

mtd_cols = ['Responsible partner','Analysis method standard', 'Analysis method name','Analysis method work instruction number', 'Analysis method source','Method specification']
df['method'] = df[mtd_cols].apply(concatenate_row_values, axis=1)



cols = ['Numerical value','Measured numerical value','Value unit','Measured value unit',
        'Monitoring location x L-EST97','Monitoring location y L-EST97','time', 'depth',
        'method','Indicator name']
df =  df[cols]


In [3]:
df['Indicator name'].unique().tolist()

['Huumussisaldus',
 'Boor',
 'Mangaan',
 'Vask',
 'Magneesium',
 'Kaltsium',
 'Kaalium',
 'Fosfor',
 'pH',
 'Horisondi tüsedus',
 'Lasuvustihedus',
 'Kaltsiumkarbonaat',
 'Üldlämmastik',
 'Üldorgaaniline süsinik',
 'Osaproovide arv proovis',
 'Reaktiivne raud',
 'Reaktiivne alumiinium',
 'Väävel',
 'Naatrium',
 'Elavhõbe',
 'Raud',
 'Alumiinium',
 'Asendusmagneesium',
 'Asenduskaltsium',
 'Tsink',
 'Nikkel',
 'Plii',
 'Aktiivne happesus',
 'Asendushappesus',
 'Indeno(1,2,3-cd)püreen',
 'Benso(g,h,i)perüleen',
 'Heptakloor',
 'Antratseen',
 'Fluoranteen',
 'Huumusvaru',
 'Lõimise liiva fraktsioon',
 'pH (CaCl2)',
 'Juurte sügavus',
 'Diagnostilise tunnuse 5 sügavus',
 'Diagnostilise tunnuse 4 sügavus',
 'Diagnostilise tunnuse 3 sügavus',
 'Diagnostilise tunnuse 2 sügavus',
 'Diagnostilise tunnuse 1 sügavus',
 'Korese maht',
 'Korese kaal',
 'Kroom',
 'Asendusalumiinium',
 'Asenduskaalium',
 'Karbonaadid',
 'Küllastusaste',
 'pH (H2O)',
 'Lõimise tolmu fraktsioon',
 'Lõimise savi fraktsi

In [3]:
para_list = []
info = ['Numerical value','Measured numerical value','Value unit','Measured value unit']
# create a function to examine the soil properties one by one
def exam_para(para_name,para_analy):
    temp = df.loc[df['Indicator name']==para_analy]
    print(f'{para_name}-{para_analy} unit: ',temp['Measured value unit'].unique().tolist()+temp['Value unit'].unique().tolist())

    temp = temp.rename(columns={'method':f'{para_name}_method'})

    no_nan_num = len(temp)-temp[temp['Numerical value'].isna() & temp['Measured numerical value'].isna()].shape[0]
    method_record_num = temp[f'{para_name}_method'].notna().sum()
    
    print(f'{len(temp)} data records in total, {no_nan_num} with valid {para_name} values, {method_record_num} with measurement method recorded')
    temp_method = temp.loc[~temp[f'{para_name}_method'].isna()].reset_index(drop=True)

    # print('method numbers overview:')
    num = []
    name = []
    for i in temp_method[f'{para_name}_method'].unique().tolist():
        method_num = sum(sum([temp_method[f'{para_name}_method']==i]))
        name.append(i)
        num.append(method_num)
        # print(f'{i} - {method_num}')
    
    return tuple(zip(num,name)),temp

def print_overview(nnlist,filt=0):
    if filt==0:
        print('overview:')
        for i in range(len(nnlist)):
            print(f'{nnlist[i][1]} - {nnlist[i][0]}')
    else:
        print(f'overview with filter {filt}:')
        for i in range(len(nnlist)):
            if filt in nnlist[i][1]:
                print(f'{nnlist[i][1]} - {nnlist[i][0]}')

In [4]:
# oc
para_name = 'oc'
para_analy = 'Üldorgaaniline süsinik'
nnlist, temp = exam_para(para_name,para_analy)
print_overview(nnlist)
temp[para_name] = temp[info[0]]/1000 # mg/kg -> g/kg
temp[f'{para_name}_unit'] = 'g/kg'

print()
print(f'{len(temp)} {para_name} values left')
para_list.append(temp)

oc-Üldorgaaniline süsinik unit:  ['g/kg KA', 'mg/kg KA']
1570 data records in total, 1570 with valid oc values, 1570 with measurement method recorded
overview:
Metsakaitse- ja Metsauuenduskeskus--ISO 10694--ISO 10694 (IR) - 1126
Keskkonnaagentuur--ISO 10694--ISO 10694 (IR) - 444

1570 oc values left


In [5]:
# oc
para_name = 'oc'
para_analy = 'Orgaaniline süsinik'
nnlist, temp = exam_para(para_name,para_analy)
print_overview(nnlist)
temp[para_name] = temp[info[0]]*10 # % -> g/kg
temp[f'{para_name}_unit'] = 'g/kg'

temp.loc[temp[f'{para_name}_method']==nnlist[1][1],para_name] = temp.loc[temp[f'{para_name}_method']==nnlist[1][1],para_name]*1.15

print()
print(f'{len(temp)} {para_name} values left')
para_list.append(temp)

oc-Orgaaniline süsinik unit:  ['% KA', '% KA']
58 data records in total, 58 with valid oc values, 58 with measurement method recorded
overview:
Põllumajandusuuringute Keskus--ISO 10694--Dumas'e meetod; ISO 10694 (GC)--Dumas'e meetod ehk kuivtuhastamise meetod (dry combustion) - 51
Põllumajandusuuringute Keskus--GOST 26213--Tjurin'i meetod; GOST 26213 (TITR)--Ponomariova VV, Plotnikova TA (1980) Humus and soil formation (Methods and results). Nauka, Moscow.  - 7

58 oc values left


In [6]:
# oc
para_name = 'oc'
para_analy = 'Huumussisaldus'
nnlist, temp = exam_para(para_name,para_analy)
print_overview(nnlist)
temp[para_name] = temp[info[0]]*10/1.724 # humus % -> oc g/kg
temp[f'{para_name}_unit'] = 'g/kg'

convert_list = [i[1] for i in nnlist if 'GOST' in i[1]]
temp = temp.loc[temp[f'{para_name}_method'].isin(convert_list)] 
temp[para_name] = temp[para_name]*1.15

print()
print(f'{len(temp)} {para_name} values left')
para_list.append(temp)

oc-Huumussisaldus unit:  ['% KA', '% KA']
185 data records in total, 185 with valid oc values, 185 with measurement method recorded
overview:
Põllumajandusuuringute Keskus--GOST 26213--Tjurin'i meetod; GOST 26213 (TITR)--Ponomariova VV, Plotnikova TA (1980) Humus and soil formation (Methods and results). Nauka, Moscow.  - 179
Põllumajandusuuringute Keskus - 6

179 oc values left


In [7]:
# # caco3
# para_name = 'caco3'
# para_analy = 'Kaltsiumkarbonaat'
# nnlist, temp = exam_para(para_name,para_analy)
# print_overview(nnlist)
# temp[para_name] = temp[info[0]]
# temp[f'{para_name}_unit'] = 'g/kg'

# print()
# print(f'{len(temp)} {para_name} values left')
# para_list.append(temp)

In [8]:
# N
para_name = 'N'
para_analy = 'Üldlämmastik'
nnlist, temp = exam_para(para_name,para_analy)
print_overview(nnlist)
temp[para_name] = temp[info[0]]/1000 # mg/kg -> g/kg
temp[f'{para_name}_unit'] = 'g/kg'

print()
print(f'{len(temp)} {para_name} values left')
para_list.append(temp)

N-Üldlämmastik unit:  ['g/kg KA', 'mg/kg KA']
1793 data records in total, 1793 with valid N values, 1793 with measurement method recorded
overview:
Metsakaitse- ja Metsauuenduskeskus--ISO 11261--Kjeldahl-i meetod ISO 11261 (TITR) - 1349
Keskkonnaagentuur--ISO 11261--Kjeldahl-i meetod ISO 11261 (TITR) - 444

1793 N values left


In [9]:
# ph-kcl, converting to ph-cacl2
para_name = 'ph_cacl2'
para_analy = 'pH'
nnlist, temp = exam_para(para_name,para_analy)
print_overview(nnlist)

temp = temp.loc[temp[f'{para_name}_method']=='Põllumajandusuuringute Keskus--ISO 10390--pH(KCl) ISO 10390 (EC)']
temp[para_name] = temp[info[0]]*0.987+0.321
temp = temp.loc[temp[para_name]<13]
temp[f'{para_name}_unit'] = '-'

print()
print(f'{len(temp)} {para_name} values left')
para_list.append(temp)

ph_cacl2-pH unit:  [nan, nan]
177 data records in total, 177 with valid ph_cacl2 values, 177 with measurement method recorded
overview:
Põllumajandusuuringute Keskus--ISO 10390--pH(KCl) ISO 10390 (EC) - 147
Põllumajandusuuringute Keskus--ISO 10390--ISO 10390 (EC) - 30

118 ph_cacl2 values left


In [10]:
# ph-cacl2
para_name = 'ph_cacl2'
para_analy = 'pH (CaCl2)'
nnlist, temp = exam_para(para_name,para_analy)
print_overview(nnlist)

temp[para_name] = temp[info[0]]
temp = temp.loc[temp[para_name]<12]
temp[f'{para_name}_unit'] = '-'

print()
print(f'{len(temp)} {para_name} values left')
para_list.append(temp)

ph_cacl2-pH (CaCl2) unit:  [nan, nan]
68 data records in total, 68 with valid ph_cacl2 values, 68 with measurement method recorded
overview:
Metsakaitse- ja Metsauuenduskeskus--ISO 10390--ISO 10390 (EC) - 38
Keskkonnaagentuur--ISO 10390--ISO 10390 (EC) - 30

68 ph_cacl2 values left


In [11]:
# clay
para_name = 'clay'
para_analy = 'Lõimise savi fraktsioon'
nnlist, temp = exam_para(para_name,para_analy)
print_overview(nnlist)

temp[para_name] = temp[info[0]]
temp[f'{para_name}_unit'] = '%'

print()
print(f'{len(temp)} {para_name} values left')
para_list.append(temp)

clay-Lõimise savi fraktsioon unit:  ['% KA', '% KA']
43 data records in total, 43 with valid clay values, 43 with measurement method recorded
overview:
Metsakaitse- ja Metsauuenduskeskus--ISO 11277--Mulla lõimise määramine - ISO 11277 - 22
Põllumajandusuuringute Keskus--Lõimise määramine (EMÜ mullalabor)--Reitam ja Penu 2009; van Reeuwijk 2002 - 21

43 clay values left


In [12]:
# sand
para_name = 'sand'
para_analy = 'Lõimise liiva fraktsioon'
nnlist, temp = exam_para(para_name,para_analy)
print_overview(nnlist)

temp[para_name] = temp[info[0]]
temp[f'{para_name}_unit'] = '%'

print()
print(f'{len(temp)} {para_name} values left')
para_list.append(temp)

sand-Lõimise liiva fraktsioon unit:  ['% KA', '% KA']
62 data records in total, 62 with valid sand values, 62 with measurement method recorded
overview:
Metsakaitse- ja Metsauuenduskeskus--ISO 11277--Mulla lõimise määramine - ISO 11277 - 34
Põllumajandusuuringute Keskus--Lõimise määramine (EMÜ mullalabor)--Reitam ja Penu 2009; van Reeuwijk 2002 - 28

62 sand values left


In [13]:
# silt
para_name = 'silt'
para_analy = 'Lõimise tolmu fraktsioon'
nnlist, temp = exam_para(para_name,para_analy)
print_overview(nnlist)

temp[para_name] = temp[info[0]]
temp[f'{para_name}_unit'] = '%'

print()
print(f'{len(temp)} {para_name} values left')
para_list.append(temp)

silt-Lõimise tolmu fraktsioon unit:  ['% KA', '% KA']
54 data records in total, 54 with valid silt values, 54 with measurement method recorded
overview:
Metsakaitse- ja Metsauuenduskeskus--ISO 11277--Mulla lõimise määramine - ISO 11277 - 26
Põllumajandusuuringute Keskus--Lõimise määramine (EMÜ mullalabor)--Reitam ja Penu 2009; van Reeuwijk 2002 - 28

54 silt values left


In [14]:
# # extractable P
# para_name = 'P'
# para_analy = 'Fosfor'
# nnlist, temp = exam_para(para_name,para_analy)
# print_overview(nnlist)

# temp =temp.loc[~temp[f'{para_name}_method'].isin(['Metsakaitse- ja Metsauuenduskeskus--ISO 11466--ISO 11466 (ICP-OES)',
#                                                  'Keskkonnaagentuur--ISO 11466--ISO 11466 (ICP-OES)'])]

# mehlich = 'Põllumajandusuuringute Keskus--Mehlich III meetod (ICP-OES)--Mehlich A (1984) Mehlich 3 soil extractant: A modification of Mehlich 2 extractant.  Commun. in Soil Science and Plant Analysis 15:1409-1416'
# temp.loc[temp[f'{para_name}_method'] == mehlich, para_name] = temp.loc[temp[f'{para_name}_method'] == mehlich, para_name] * 0.46 + 1.5
# dl = "Põllumajandusuuringute Keskus--GOST 26209--Egner-Riehm'i DL-meetod; GOST 26209 (ICP-OES)--Egnér H, Riehm H, Domingo WR, 1960. Chemische Extraktionsmethoden zur Phosphor und Kaliumbestimmung. Kungliga Lantbrukshögskolans annaler 26:199-215"
# temp.loc[temp[f'{para_name}_method'] == dl, para_name] = temp.loc[temp[f'{para_name}_method'] == dl, para_name]*0.262 + 25.2

# temp[para_name] = temp[info[0]]
# temp[f'{para_name}_unit'] = 'mg/kg'

# print()
# print(f'{len(temp)} {para_name} values left')
# para_list.append(temp)

In [15]:
# extractable K
para_name = 'K'
para_analy = 'Kaalium'
nnlist, temp = exam_para(para_name,para_analy)
print_overview(nnlist)

temp =temp.loc[~temp[f'{para_name}_method'].isin(['Metsakaitse- ja Metsauuenduskeskus--ISO 11466--ISO 11466 (ICP-OES)',
                                                  'Keskkonnaagentuur--ISO 11466--ISO 11466 (ICP-OES)',
                                                  'Põllumajandusuuringute Keskus--STJnr.M/U91 (ICP-OES)--STJnr.M/U91--Vee,muda ja settee metallisisalduste määramine induktiivsisestunud plasma aatomemissioonspektromeetriga.'])]

temp[para_name] = temp[info[0]]
temp[f'{para_name}_unit'] = 'mg/kg'

mehlich = 'Põllumajandusuuringute Keskus--Mehlich III meetod (ICP-OES)--Mehlich A (1984) Mehlich 3 soil extractant: A modification of Mehlich 2 extractant.  Commun. in Soil Science and Plant Analysis 15:1409-1416'
temp.loc[temp[f'{para_name}_method'] == mehlich, para_name] = temp.loc[temp[f'{para_name}_method'] == mehlich, para_name] * 0.965 + 7.13
dl = "Põllumajandusuuringute Keskus--GOST 26209--Egner-Riehm'i DL-meetod; GOST 26209 (ICP-OES)--Egnér H, Riehm H, Domingo WR, 1960. Chemische Extraktionsmethoden zur Phosphor und Kaliumbestimmung. Kungliga Lantbrukshögskolans annaler 26:199-215"
temp.loc[temp[f'{para_name}_method'] == dl, para_name] = (temp.loc[temp[f'{para_name}_method'] == dl, para_name] + 14.6525) / 1.1041

print()
print(f'{len(temp)} {para_name} values left')
para_list.append(temp)

K-Kaalium unit:  ['mg/kg KA', 'mg/kg KA']
4220 data records in total, 4220 with valid K values, 4220 with measurement method recorded
overview:
Põllumajandusuuringute Keskus--Mehlich III meetod (ICP-OES)--Mehlich A (1984) Mehlich 3 soil extractant: A modification of Mehlich 2 extractant.  Commun. in Soil Science and Plant Analysis 15:1409-1416 - 1654
Põllumajandusuuringute Keskus--GOST 26209--Egner-Riehm'i DL-meetod; GOST 26209 (ICP-OES)--Egnér H, Riehm H, Domingo WR, 1960. Chemische Extraktionsmethoden zur Phosphor und Kaliumbestimmung. Kungliga Lantbrukshögskolans annaler 26:199-215 - 1216
Metsakaitse- ja Metsauuenduskeskus--ISO 11466--ISO 11466 (ICP-OES) - 804
Põllumajandusuuringute Keskus--STJnr.M/U91 (ICP-OES)--STJnr.M/U91--Vee,muda ja settee metallisisalduste määramine induktiivsisestunud plasma aatomemissioonspektromeetriga. - 392
Keskkonnaagentuur--ISO 11466--ISO 11466 (ICP-OES) - 154

2870 K values left


In [16]:
# bulk density
para_name = 'bulk_density'
para_analy = 'Lasuvustihedus'
nnlist, temp = exam_para(para_name,para_analy)
print_overview(nnlist)

temp[para_name] = temp[info[1]]
temp.loc[temp['Measured value unit']=='kg/m³ KA',para_name] = temp.loc[temp['Measured value unit']=='kg/m³ KA',para_name]/1000
temp[f'{para_name}_unit'] = 'g/cm³'
temp = temp.loc[temp['bulk_density']<10]  # keep valid values
print()
print(f'{len(temp)} {para_name} values left')
para_list.append(temp)

bulk_density-Lasuvustihedus unit:  ['g/cm³ KA', 'kg/m³ KA', 'g/cm³ KA']
114 data records in total, 114 with valid bulk_density values, 114 with measurement method recorded
overview:
Põllumajandusuuringute Keskus--Mulla lasuvustiheduse määramine (EMÜ mullalabor)--Reitam E, Penu P (2009) Eesti põllumuldade tallatuse seire 2008. aastal. Eesti Maaülikool, Põllumajandusuuringute Keskus. Kättesaadav http://seire.keskkonnainfo.ee/attachments/article/1631/Aruanne_2.doc (vaadatud detsembris 2015) - 7
Metsakaitse- ja Metsauuenduskeskus--ISO 11272--Mulla lasuvustiheduse määramine - ISO 11272 - 107

108 bulk_density values left


In [17]:
# CEC
para_name = 'CEC'
para_analy = 'Katioonvahetusvõime'
nnlist, temp = exam_para(para_name,para_analy)
print_overview(nnlist)

temp[para_name] = temp[info[0]]
temp[f'{para_name}_unit'] = 'cmol+/kg'
print()
print(f'{len(temp)} {para_name} values left')
para_list.append(temp)

CEC-Katioonvahetusvõime unit:  ['cmol+/kg KA', 'cmol+/kg KA']
3 data records in total, 3 with valid CEC values, 3 with measurement method recorded
overview:
Metsakaitse- ja Metsauuenduskeskus--ISO 11260--ISO 11260 & ISO 14254 (ICP-OES) - 3

3 CEC values left


In [23]:
final = pd.concat(para_list)
ulist = [i for i in final.columns.tolist() if 'unit' in i or 'method' in i] + ['Numerical value', 'Measured numerical value','Value unit', 'Measured value unit']
final = final.drop(columns=ulist)

merge = (final.groupby(['Monitoring location x L-EST97', 'Monitoring location y L-EST97','time', 'depth'])
            .agg({
                # 'ph_kcl': 'mean',
                'ph_cacl2': 'mean',
                'oc': 'mean',
                'sand': 'mean',
                'clay': 'mean',
                'silt': 'mean',
                'N': 'mean',
                # 'P': 'mean',
                'K': 'mean',
                # 'caco3': 'mean',
                'bulk_density':'mean',
                'CEC':'mean'
                # 'ph_H2O_SOIL_ANALYSIS_METHOD_REFERENCE': 'first',
                # 'ph_CaCl2_SOIL_ANALYSIS_METHOD_REFERENCE': 'first',
                # 'ph_KCl_SOIL_ANALYSIS_METHOD_REFERENCE': 'first',
                # 'oc_SOIL_ANALYSIS_METHOD_REFERENCE': 'first',
                # 'Sand_SOIL_ANALYSIS_METHOD_REFERENCE': 'first',
                # 'Ton_SOIL_ANALYSIS_METHOD_REFERENCE': 'first',
                # 'Schluff_SOIL_ANALYSIS_METHOD_REFERENCE': 'first',
                # 'N_SOIL_ANALYSIS_METHOD_REFERENCE': 'first',
                # 'P_SOIL_ANALYSIS_METHOD_REFERENCE': 'first',
                # 'K_SOIL_ANALYSIS_METHOD_REFERENCE': 'first',
                # 'Kalk_SOIL_ANALYSIS_METHOD_REFERENCE': 'first'
            })
            .reset_index())

merge['hzn_top'] = merge['depth'].str.split(' -').str[0]
merge['hzn_btm'] = merge['depth'].str.split(' -').str[-1]
merge.loc[merge['hzn_top']=='Huumushorisondist (A)','hzn_top'] = 0
merge.loc[merge['hzn_btm']=='Huumushorisondist (A)','hzn_btm'] = 20
merge.loc[merge['hzn_top']=='Künnikihist (5 cm sügavuselt) ja künnikihi all olevast kihist (20 cm sügavuselt)','hzn_top'] = 5
merge.loc[merge['hzn_btm']=='Künnikihist (5 cm sügavuselt) ja künnikihi all olevast kihist (20 cm sügavuselt)','hzn_btm'] = 30
merge['hzn_top'] = merge['hzn_top'].apply(pd.to_numeric,errors='coerce')
merge['hzn_btm'] = merge['hzn_btm'].apply(pd.to_numeric,errors='coerce')

import pyproj
import pandas as pd

lest97 = pyproj.Proj(projparams='epsg:3301')  # EPSG code for L-EST97
wgs84 = pyproj.Proj(projparams='epsg:4326')  # EPSG code for WGS 84

merge['Monitoring location x L-EST97'] = merge['Monitoring location x L-EST97'].str.replace(",", ".").apply(pd.to_numeric,errors='coerce')
merge['Monitoring location y L-EST97'] = merge['Monitoring location y L-EST97'].str.replace(",", ".").apply(pd.to_numeric,errors='coerce')

merge['lat'], merge['lon'] = pyproj.transform(lest97, wgs84, merge['Monitoring location x L-EST97'], merge['Monitoring location y L-EST97'])

/tmp/ipykernel_1593/3078699769.py:51: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  merge['lat'], merge['lon'] = pyproj.transform(lest97, wgs84, merge['Monitoring location x L-EST97'], merge['Monitoring location y L-EST97'])


In [24]:
merge = merge[['lat','lon','time','hzn_top', 'hzn_btm', 'ph_cacl2', 'oc', 
               'sand', 'clay', 'silt','N', 'K', 'bulk_density']]  #'P', 'caco3', 'ph_kcl'


merge = merge.dropna(subset = ['lat','lon','time','hzn_top', 'hzn_btm'],how = 'any')

In [25]:
merge.to_csv('/home/opengeohub/xuemeng/work_xuemeng/spatiotemporal-soc-eu/estonia/estonia_harmonized_v1.csv',index=False)

In [26]:
merge

,lat,lon,time,hzn_top,hzn_btm,ph_cacl2,oc,sand,clay,silt,N,K,bulk_density
0,57.950126,26.437069,2004.0,0,20,NaN,NaN,NaN,NaN,NaN,NaN,41.000000,NaN
1,57.950185,26.436922,2009.0,0,20,6.243,NaN,NaN,NaN,NaN,NaN,53.000000,NaN
2,57.950210,26.436884,2014.0,0,20,NaN,NaN,NaN,NaN,17.0,NaN,77.835522,NaN
3,57.950214,26.437097,2004.0,0,20,NaN,NaN,NaN,NaN,NaN,NaN,51.000000,NaN
4,57.950272,26.436964,2009.0,0,20,NaN,NaN,NaN,NaN,NaN,NaN,36.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338,59.307325,26.400573,2009.0,0,20,NaN,NaN,NaN,NaN,NaN,NaN,266.925596,NaN
1339,59.307342,26.400705,2014.0,0,20,NaN,NaN,NaN,NaN,NaN,NaN,401.000000,NaN
1340,59.307355,26.400229,2004.0,0,20,NaN,NaN,NaN,NaN,NaN,NaN,377.000000,NaN
1341,59.307413,26.400303,2014.0,0,20,NaN,30.0,NaN,NaN,NaN,NaN,423.231365,NaN
